<a href="https://colab.research.google.com/github/bebyakinb/test_assignments_Analyst/blob/master/Saber/Task_1_and_2_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. Подготовка среды

##0.1. Загрузка актуальной версии SQLite3

In [7]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz &> /dev/null
!tar xzf sqlite.tar.gz &> /dev/null
%cd sqlite/
!./configure &> /dev/null
!make sqlite3.c &> /dev/null
%cd /content
!npx degit coleifer/pysqlite3 -f &> /dev/null
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build &> /dev/null
!cp build/lib.linux-x86_64-3.8/pysqlite3/_sqlite3.cpython-38-x86_64-linux-gnu.so \
    /usr/lib/python3.8/lib-dynload/_sqlite3.cpython-38-x86_64-linux-gnu.so

/content/sqlite
/content


## 0.2. Загрузка БД для работы с заданиями

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1t3LaSwQPqp8OdXKiAPdZZ1YmhaGxpDfj' -O test.db  &> /dev/null

## 0.3. Загрузка библиотек

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.40.1'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##0.4. Подключение к БД

In [3]:
def select(sql):
  return pd.read_sql(sql,con)

In [4]:
con = sqlite3.connect('test.db')
cur = con.cursor()

In [5]:
sql= """
select
    name
from
    sqlite_schema
where 
    type ='table' and 
    name not like 'sqlite_%';
"""
select(sql)



,name
0,history


# 1. Задача №1

##1.0. Условия задачи

Напишите запрос, который выведет, сколько времени в среднем задачи каждой группы находятся в статусе “Open”

Условия:

Под группой подразумевается первый символ в ключе задачи. Например, для ключа “C-40460” группой будет “C”

Задача может переходить в один и тот же статус несколько раз.

Переведите время в часы с округлением до двух знаков после запятой.


## 1.1. Решение

In [6]:
sql = """
select
    substr(issue_key, 1, 1) as working_group,
    round(avg(avg_minutes)/60,2) as avg_hours
from
    (select 
        issue_key,
        sum(minutes_in_status) as avg_minutes
    from 
        history
    where
        status = 'Open'
    group by
        issue_key
     )
group by
    working_group
"""
select(sql)

,working_group,avg_hours
0,A,247.90
1,B,281.51
2,C,595.62
3,D,842.02
4,E,817.94


# 2. Задача №2

## 2.0. Постановка задачи

Напишите запрос, который выведет ключ задачи, последний статус и его время создания для задач, которые открыты на данный момент времени.

Условия:

Открытыми считаются задачи, у которых последний статус в момент времени не “Closed” и не “Resolved”

Задача может переходить в один и тот же статус несколько раз.

Оформите запрос таким образом, чтобы, изменив дату, его можно было использовать для поиска открытых задач в любой момент времени в прошлом

Переведите время в текстовое представление


## 2.1. Решение

In [11]:
sql = """
select 
    issue_key,
    status,
    strftime('%Y-%m-%d %H:%M:%S', started_at / 1000, 'unixepoch') as start_time
from
    history
where
    (julianday('now') - 2440587.5)*86400.0*1000.0 between started_at and ended_at
    and status <> "Closed"
    and status <> "Resolved"
"""
select(sql)

,issue_key,status,start_time


Для поиска открытых задач в прошлом нужно заменить 'now' на нужную дату, как в примере ниже:

In [10]:
sql = """
select 
    issue_key,
    status,
    strftime('%Y-%m-%d %H:%M:%S', started_at / 1000, 'unixepoch') as start_time
from
    history
where
    (julianday('2022-11-26') - 2440587.5)*86400.0*1000.0 between started_at and ended_at
    and status <> "Closed"
    and status <> "Resolved"
"""
select(sql)

,issue_key,status,start_time
0,C-40460,In Progress,2022-11-25 10:04:00
1,C-38984,Reopened,2022-11-15 10:15:23
2,C-37862,In Review,2022-11-14 08:59:04
3,B-4645,Reopened,2022-11-25 09:23:14
4,C-40059,In Progress,2022-11-23 09:16:30
5,C-38787,In Owl,2022-11-25 07:41:16
6,C-36721,In Owl,2022-11-18 11:45:00
7,C-35057,Reopened,2022-11-14 11:49:52
8,C-31154,In Owl,2022-09-14 17:02:17
9,A-52884,In Progress,2022-11-25 10:52:57


P.S. Для удобства в запрос можно было бы добавить переменную для даты, но sqlite не поддерживает такой синтаксис, альтернатива в виде временной таблицы выглядят громоздкой для этой задачи